### Libraries

In [1]:
!pip install -qU langchain-community faiss-cpu
!pip install -qU langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 850.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5

In [2]:
import pandas as pd

### CSV file transfroming to Pandas dataframe

In [9]:
df = pd.read_csv("goods.csv", index_col=0)
df_unique = df.drop_duplicates("name")

Dataframe structure

In [10]:
df_unique.sample(4)

,name,category,price,rating,url,description
8879,Наклейки для цветов и подарков «Ты лучше всех»...,Флористика,7.5,5.0,https://www.sima-land.ru/9224777/nakleyki-dlya...,Декоративные наклейки - это яркое и веселое ре...
4908,"Набор резцов универсальных ЛОМ, пластиковая ру...",Инструменты,199.0,3.8,https://www.sima-land.ru/2303624/nabor-rezcov-...,Набор универсальных резцов ЛОМ с пластиковой р...
906,"Лента репсовая с тиснением «Сердечки», 25 мм, ...",Ленты,259.0,5.0,https://www.sima-land.ru/9327064/lenta-repsova...,"Те, кто увлекаются рукоделием, отлично знакомы..."
14125,"Наклейка для творчества пластик ""Море"" 6,5х19 см",Принадлежности для творчества,9.3,5.0,https://www.sima-land.ru/9863178/nakleyka-dlya...,NaN


In [11]:
print(df_unique.shape[0])                                       # number of items
print(df_unique[df_unique['description'].isna()].shape[0])      # number of items without description

14615
5447


In [12]:
df_unique.to_csv("goods_unique.csv", sep='\t')

### Formatting Pandas dataframe to Langchain Documents

In [13]:
from langchain_core.documents import Document


docs = []

for row in df_unique[~df_unique['description'].isna()].itertuples():
    doc = Document(
        page_content=f"Название: {row.name}. Описание: {row.description}",
        metadata={
            "url": row.url,
            "category": row.category,
            "rating": row.rating,
            "price": row.price,
            "name": row.name
        },
    )
    docs.append(doc)

for row in df_unique[df_unique['description'].isna()].itertuples():
    doc = Document(
        page_content=f"Название: {row.name}. Описание: описание отсутствует",
        metadata={
            "url": row.url,
            "category": row.category,
            "rating": row.rating,
            "price": row.price,
            "name": row.name
        },
    )
    docs.append(doc)

FAISS index creation

In [14]:
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings


embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

db = FAISS.from_documents(docs, embeddings)
db.save_local("faiss_index")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.89k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Some examples

In [15]:
results = db.similarity_search("Люблю рисовать", 5)
for res in results:
    print(res.metadata['name'])

Картина по номерам для детей «Игровой мир», 20 х 30 см
Картина по номерам панно «Кот на руках», 30 х 50 см
Картина по номерам на холсте с подрамником «Искусство», 40 х 50 см
Картина по номерам на холсте с подрамником «Девушка», 40 х 50 см
Картина по номерам для детей «Волшебство в руках», 20 х 30 см


In [16]:
results = db.similarity_search("Люблю шить", 5)
for res in results:
    print(res.metadata['name'])

Спицы для вязания, прямые, d = 5 мм, 25 см, 2 шт
Лапка для пришивания резинки, 4,8 × 2,5 см
Шило швейное, 6,5 × 1 см, цвет МИКС
Иглы для бытовых швейных машин, для стрейч-ткани, №70-100, 3 шт
Спицы для вязания, прямые, d = 4 мм, 35 см, 2 шт


In [17]:
results = db.similarity_search("Люблю музыкальные инструменты", 5)
for res in results:
    print(res.metadata['name'])

Сувенирный музыкальный инструмент Бубен «Уникальность», 23×23×7,5 см
Звукосниматель для музыкальных инструментов Music Life
Сувенирный музыкальный инструмент "Калимба узор" круглая 7х14х14 см
Набор аксессуаров для акустической гитары
Музыкальный инструмент Тарелки ручные Music Life 9 см
